In [57]:
import pandas as pd

In [70]:
import csv

with open("data/english/syllable_frequency.cd", "r", encoding='utf-8') as csv_file:
        fdata = list(csv.reader(csv_file))

for i, syllable, position, frequency_pos_1m, frequency_total_1m in fdata[1:]:
        i, position, frequency_pos_1m, frequency_total_1m = \
                int(i), int(position), int(frequency_pos_1m), int(frequency_total_1m)
        print(syllable, position, frequency_pos_1m, frequency_total_1m)
print("Total: ", i+1)

ɑː 1 1003 1070
ɑː 2 59 1070
ɑː 3 8 1070
ɑː 4 0 1070
ɑː 5 0 1070
ɑː 7 0 1070
ɑːɲ 1 25 25
ɑːɲ 3 0 25
ɑːɲ 5 0 25
ɑːɲt 1 5 5
ɑːɜ˞ 1 4672 4688
ɑːɜ˞ 3 3 4688
ɑːɜ˞ 4 13 4688
ɑːʃ 2 0 0
ɑːf 1 1344 1344
ɑːf 2 0 1344
ɑːf 3 0 1344
ɑːf 4 0 1344
ɑːf 5 0 1344
ɑːft 1 1 1
ɑːk 1 18 20
ɑːk 3 2 20
ɑːks 1 1 1
ɑːks 3 0 1
ɑːkt 1 0 0
ɑːl 2 0 0
ɑːl 5 0 0
ɑːlz 2 0 0
ɑːm 1 130 135
ɑːm 2 5 135
ɑːmd 1 38 41
ɑːmd 2 3 41
ɑːmz 1 149 149
ɑːmz 2 0 149
ɑːmz 3 0 149
ɑːmz 4 0 149
ɑːmz 5 0 149
ɑːn 1 313 319
ɑːn 2 6 319
ɑːn 3 0 319
ɑːns 2 1 1
ɑːnst 2 0 0
ɑːnt 1 106 106
ɑːnt 2 0 106
ɑːnts 1 4 4
ɑːnts 2 0 4
ɑːnz 2 0 0
ɑːs 1 5 5
ɑːs 4 0 5
ɑːsk 1 224 224
ɑːsks 1 44 44
ɑːskt 1 465 466
ɑːskt 2 1 466
ɑːst 1 0 0
ɑːt 1 168 168
ɑːt 2 0 168
ɑːt 4 0 168
ɑːts 1 46 46
ɑːts 2 0 46
ɑːts 4 0 46
ɑːv 2 0 0
ɑːvz 2 0 0
ɑːz 1 1 1
ɑːz 2 0 1
ɔː 1 1662 1732
ɔː 2 68 1732
ɔː 3 2 1732
ɔː 4 0 1732
ɔː 5 0 1732
ɔː 6 0 1732
ɔːɜ˞ 1 4271 4273
ɔːɜ˞ 2 0 4273
ɔːɜ˞ 3 1 4273
ɔːɜ˞ 4 1 4273
ɔːb 1 0 0
ɔːbz 1 0 0
ɔːd 1 6 6
ɔːɡ 1 4 4
ɔːɡ 3 0 4
ɔːk 1 8 8
ɔːk 2 0 8
ɔːk

In [13]:
from arc.core.syllable import syllable_from_phonemes
from arc.io import *


def read_syllables_dict_deu_special(check_phonemes: bool = True):
    assert check_phonemes == False, "check phonemes not supported yet for german corpus"
    with open(SYLLABLES_DEFAULT_PATH_DEU_SPECIAL, "r", encoding='utf-8') as csv_file:
        fdata = list(csv.reader(csv_file, delimiter='\t'))

    syllables_dict: Dict[str, Syllable] = {}
    feature_phonemes = load_default_phonemes()

    for syll_stats in fdata[1:]:
        syll_ipa = phonecodes.xsampa2ipa(syll_stats[1], "deu")
        info = {"freq": int(syll_stats[2]), "prob": float(syll_stats[3])}
        if syll_ipa in syllables_dict and syllables_dict[syll_ipa].info != info:
            logger.warning(
                f"Syllable '{syll_ipa}' with conflicting stats. Keeping new stats."
            )
        if check_phonemes:
            try:
                phonemes = [phonecodes.disc2ipa(phon, "eng") for phon in syll_stats[0]]
                syllables_dict[syll_ipa] = syllable_from_phonemes(feature_phonemes, phoneme_combination=phonemes)
            except KeyError:
                continue
        else:
            syllables_dict[syll_ipa] = Syllable(id=syll_ipa, phonemes=[], info=info, binary_features=[], phonotactic_features=[])
    
    return syllables_dict


def read_syllables_dict_celex(lang: str, check_phonemes: bool =True):
    path = {"deu": SYLLABLES_DEFAULT_PATH_DEU, "eng": SYLLABLES_DEFAULT_PATH_ENG}.get(lang)

    with open(path, "r", encoding='utf-8') as csv_file:
        fdata = list(csv.reader(csv_file, delimiter='\\'))
    
    syllables_dict: Dict[str, Syllable] = {}
    feature_phonemes = load_default_phonemes()

    for syll_stats in fdata[1:]:
        syll_ipa = phonecodes.disc2ipa(syll_stats[0], lang)
        info = {"freq": int(syll_stats[-1])}
        if syll_ipa in syllables_dict and syllables_dict[syll_ipa].info != info:
            logger.info(
                f"Syllable '{syll_ipa}' with conflicting stats. Keeping new stats."
            )
        # we can try to take advantage of the single-character-per-phoneme property of the disc charset
        if check_phonemes:
            try:
                phonemes = [phonecodes.disc2ipa(phon, "eng") for phon in syll_stats[0]]
                syllable_object = syllable_from_phonemes(feature_phonemes, phoneme_combination=phonemes)
            except KeyError:
                continue
        else:
            syllable_object = Syllable(id=syll_ipa, phonemes=[], info=info, binary_features=[], phonotactic_features=[])

        syllables_dict[syll_ipa] = syllable_object
    
    return syllables_dict


def read_syllables_corpus(
        lang: Literal["deu", "nld", "eng"] = "deu",
        check_phonemes: bool = True,
) -> Register[str, Syllable]:
    logger.info("READ SYLLABLES, FREQUENCIES AND PROBABILITIES FROM CORPUS AND CONVERT SYLLABLES TO IPA")

    if lang in ["eng"]:
        syllables_dict = read_syllables_dict_celex(lang=lang, check_phonemes=check_phonemes)  # for german maybe: .intersection(read_syllables_dict_deu_special())
    elif lang == "deu":
        syllables_dict = read_syllables_dict_deu_special(check_phonemes=check_phonemes) 
    else:
        raise ValueError(f"Language {lang} not supported.")

    return Register(syllables_dict)

read_syllables_corpus(lang="eng", check_phonemes=True)

Register([('ɑː',
           Syllable(id='ɑː', info={'binary_features': [1, 0, 1, 0, 1, 1], 'phonotactic_features': [[]]}, phonemes=[Phoneme(id='ɑː', info={'features': ['+', '+', '-', '+', '0', '-', '-', '0', '+', '-', '-', '-', '-', '-', '-', '-', '+', '+', '-', '+', '+']})])),
          ('ɑːɲ',
           Syllable(id='ɑːɲ', info={'binary_features': [1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1], 'phonotactic_features': [[], ['son', 'oth']]}, phonemes=[Phoneme(id='ɑː', info={'features': ['+', '+', '-', '+', '0', '-', '-', '0', '+', '-', '-', '-', '-', '-', '-', '-', '+', '+', '-', '+', '+']}), Phoneme(id='ɲ', info={'features': ['-', '+', '+', '-', '-', '-', '+', '-', '+', '-', '-', '+', '-', '0', '-', '+', '-', '-', '-', '0', '-']})])),
          ('ɑːɲt',
           Syllable(id='ɑːɲt', info={'binary_features': [1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'phonotactic_features': [[], ['son', 'oth'], ['plo', 'den']]}, phonemes=[Phoneme(id='ɑː', info={'features